In [1]:
import numpy as np
import pandas as pd
import os
import os.path as opath
import datetime
import pickle

from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<div class="span5 alert alert-info">
<h3>Note book details</h3>

<p> This notebook is part 1 of 2 for data wrangling for <b>Mortgage client segmentation</b> project.</p>

<p> Notes.</p>

<ol>
<li>  The file read the multiple csv files as input for each month </li> 
<li>  The file needs to be run manually for each month as files for following reason 
    <ul>
     <li> The files are available at end of months</li> 
    <li>  Processing each month is time consuming and resource extensive process on local computer </li> 
    </ul>
</li> 
<li>  The files are pickeld and csv to use further in project . This make easier to use data in later stages</li> 
    
</ol>
</div>

In [6]:
pd.set_option('display.max_colwidth', -1)
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }'
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='df_selectdata_hmda.sa'
df_readdata = pickle.load( open( pickle_file, "rb" ) )

In [3]:
county_data='data/county_csv.csv'

df_county=pd.read_csv(county_data, index_col=False,sep=',')

In [4]:
def checkforNullCol(colname,df):
    return df[df[colname].isnull()]#[colname,homeloancol]

In [5]:
def checkforNanCol(colname,df):
    return df[np.isnan(df[colname])]

In [18]:
# based on inital data wrangling step the data will be removed for following columns
def removeDatatForNullColumns(df):
    # remove all where no geographic info available
    indexs=df[df['MSA'].isnull() & df['StateCode'].isnull() & df['CountyCode'].isnull()].index
    if(len(indexs)>0):
        df=df.drop(indexs,inplace=True)
    else:
        print('no item to drop')
    return df

In [7]:
def GetMSAData(df_selectdata):
    
    df_selectdata_msa=df_selectdata.merge(df_county,on=('StateCode','CountyCode'),how='left')
    df_selectdata_msa_finalstate=checkforNullCol('StateCode',df_selectdata_msa)

    df_selectdata_msa_finalstate=checkforNullCol('StateName',df_selectdata_msa)
    df_selectdata_msa_finalstate=df_selectdata_msa_finalstate.merge(df_state,on=('StateCode'))
    #df_selectdata_msa_finalstate[df_selectdata_msa_finalstate.StateName_x!=df_selectdata_msa_finalstate.StateName_y]['StateName_x']=df_selectdata_msa_finalstate[df_selectdata_msa_finalstate.StateName_x!=df_selectdata_msa_finalstate.StateName_y]['StateName_y']
    df_selectdata_msa_finalstate['StateName_x']=df_selectdata_msa_finalstate['StateName_y']
    print('number of state na after update: '+str(len(df_selectdata_msa_finalstate[(df_selectdata_msa_finalstate.StateName_x!=df_selectdata_msa_finalstate.StateName_y)])))
    df_selectdata_msa_finalstate=df_selectdata_msa_finalstate.drop('StateName_x',axis=1)
    _=df_selectdata_msa_finalstate.rename(columns={'StateName_y': 'StateName'}, inplace=True)
    df_selectdata_msa=df_selectdata_msa.drop(df_selectdata_msa[df_selectdata_msa.StateName.isnull()].index)
    print('no of rows drop after county merge: '+str(len(df_selectdata_msa)))
    df_out= df_selectdata_msa.append(df_selectdata_msa_finalstate,sort=False)
    print('final len of missing state(match below numer of row drop) :'+str(len(checkforNullCol(df_selectdata_msa))))
    print('final len of missing state(match above numer of row drop) :'+str(len(df_selectdata_msa_finalstate)))
    print('total rows after county join: '+str(len(df_selectdata_msa)))
    print('total row after all county and state join:'+str(len(df_out)))
    print('total rows in input frame: '+str(len(df_selectdata)))
    return df_out

In [ ]:
def createResultCol(row):
    if (row.ActionType==1 ):
        row.Result=1
    elif (row.ActionType==2 ):
        row.Result=1
    elif (row.ActionType==3 ):
        row.Result=0
    elif (row.ActionType==4 ):
        row.Result=np.NAN        
    elif (row.ActionType==5 ):
        row.Result=np.NAN
    elif (row.ActionType==6 ):
        row.Result=1
    elif (row.ActionType==7 ):
        row.Result=0
    elif (row.ActionType==8 ):
        row.Result=1        
    else:
        row.Result=np.NAN
    return row

In [8]:
for col in ['StateCode','CountyCode']:
    #df_alldata[col]=df_alldata[col].fillna(0, inplace=True)
    df_county[col]=    df_county[col].astype(np.float64)#pd.to_numeric(df_county[col], upcast='float64')

In [9]:
df_county.info()
#df_selectdata.info()
df_state=df_county.groupby(['StateCode','StateName'],  as_index=False).count().loc[:,['StateCode','StateName']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 5 columns):
StateName        3235 non-null object
StateCode        3235 non-null float64
CountyCode       3235 non-null float64
CountyName       3235 non-null object
CLASSCODEFIPS    3235 non-null object
dtypes: float64(2), object(3)
memory usage: 126.4+ KB


In [10]:
df_selectdata_msa1=GetMSAData(df_readdata)
df_selectdata_msa1=df_selectdata_msa1.apply(createResultCol,axis=1)
df_selectdata_msa1.info()#.info()#.loc[:,['year','MSA','StateCode','CountyCode','CountyCode_cenus','StateName','StateCode_cenus','CountyName']].head()

number of state na after update: 0
no of rows drop for null state: 31580344
final len of missing state(match above numer of row drop) :49682
total rows after county join: 31580344
total row after all county and state join:31630026
total rows in input frame: 31630026
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 20 columns):
Year                    int64
PropertyType            category
LoanPurpose             category
Occupancy               category
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      category
CoApplicantEthnicity    category
ApplicantRace           category
CoApplicantRace         category
ApplicantSex            category
CoApplicantSex          category
ApplicantIncome         float64
PurchaserType           category
StateName               object
CountyName              object
CLASSCO

### Process main file for state  data

In [11]:
df_selectdata_msa1.head()

,Year,PropertyType,LoanPurpose,Occupancy,LoanAmount,ActionType,MSA,StateCode,CountyCode,ApplicantEthnicity,CoApplicantEthnicity,ApplicantRace,CoApplicantRace,ApplicantSex,CoApplicantSex,ApplicantIncome,PurchaserType,StateName,CountyName,CLASSCODEFIPS
0,2015,1,3.0,1.0,222.0,3.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,115.0,0.0,AK,Matanuska-Susitna Borough,H1
1,2015,1,3.0,2.0,188.0,3.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,115.0,0.0,AK,Matanuska-Susitna Borough,H1
2,2015,1,3.0,2.0,216.0,1.0,11260.0,2.0,20.0,2.0,5.0,5.0,8.0,1.0,5.0,230.0,7.0,AK,Anchorage Municipality,H6
3,2015,1,3.0,1.0,219.0,1.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,145.0,1.0,AK,Matanuska-Susitna Borough,H1
4,2015,1,3.0,2.0,188.0,1.0,11260.0,2.0,170.0,2.0,5.0,5.0,8.0,1.0,5.0,145.0,1.0,AK,Matanuska-Susitna Borough,H1


In [19]:
df_alldata=removeDatatForNullColumns(df_selectdata_msa1)
#df_readdata.info()

no item to drop


In [20]:
print(len(df_readdata))
print(len(df_selectdata_msa1))
print(len(df_alldata))

31630026
31630026
31630026


### Pickle and save the single file for further processing 

In [33]:
#df_selecteddata.info()
picklefilename='df_selectdata_hmda_cenus.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)

In [35]:
# create pickle file for further use 
pickle.dump(df_alldata,open(picklefilename,'wb'), protocol=4)

In [24]:
# check pickle file
upickle_df = pickle.load(open(picklefilename,"rb"))

In [25]:
upickle_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 20 columns):
Year                    int64
PropertyType            category
LoanPurpose             category
Occupancy               category
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      category
CoApplicantEthnicity    category
ApplicantRace           category
CoApplicantRace         category
ApplicantSex            category
CoApplicantSex          category
ApplicantIncome         float64
PurchaserType           category
StateName               object
CountyName              object
CLASSCODEFIPS           object
dtypes: category(10), float64(6), int64(1), object(3)
memory usage: 2.9+ GB


In [26]:
df_newdata=upickle_df

In [27]:
df_newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 20 columns):
Year                    int64
PropertyType            category
LoanPurpose             category
Occupancy               category
LoanAmount              float64
ActionType              float64
MSA                     float64
StateCode               float64
CountyCode              float64
ApplicantEthnicity      category
CoApplicantEthnicity    category
ApplicantRace           category
CoApplicantRace         category
ApplicantSex            category
CoApplicantSex          category
ApplicantIncome         float64
PurchaserType           category
StateName               object
CountyName              object
CLASSCODEFIPS           object
dtypes: category(10), float64(6), int64(1), object(3)
memory usage: 2.9+ GB


In [29]:
for col in df_newdata.columns:
    
    df=checkforNullCol(col,df_newdata)
    if (len(df)>0):
        print(col + ' Null len :'+str(len(df)) )
    #df1=checkforNanCol(col)
    #if (len(df1)>0):
        #print(col + ' Nan len :'+str(len(df1)) )
        

MSA Null len :3495091
CountyCode Null len :49639
CountyName Null len :49682
CLASSCODEFIPS Null len :49682


In [30]:
df_newdata.groupby('ActionType').size() 

ActionType
1.0    21287511
2.0    1125929 
3.0    6450624 
6.0    2762256 
7.0    1952    
8.0    1754    
dtype: int64

In [32]:
print(len(df_newdata))
print(len(df_alldata))

31630026
31630026


### process main file for state

In [36]:
pickle_file='df_alldata_hmda.sa'
df_readdata = pickle.load( open( pickle_file, "rb" ) )

In [37]:
df_selectdata_msa1=GetMSAData(df_readdata)
df_selectdata_msa1=df_selectdata_msa1.apply(createResultCol,axis=1)
df_selectdata_msa1.info()#.info()#.loc[:,['year','MSA','StateCode','CountyCode','CountyCode_cenus','StateName','StateCode_cenus','CountyName']].head()

number of state na after update: 0
no of rows drop for null state: 31580344
final len of missing state(match above numer of row drop) :49682
total rows after county join: 31580344
total row after all county and state join:31630026
total rows in input frame: 31630026
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 48 columns):
Year                        int64
RespondentID                object
AgencyCode                  int64
LoanType                    int64
PropertyType                category
LoanPurpose                 category
Occupancy                   category
LoanAmount                  float64
Preapproval                 float64
ActionType                  float64
MSA                         float64
StateCode                   float64
CountyCode                  float64
CensusTractNumber           object
ApplicantEthnicity          category
CoApplicantEthnicity        category
ApplicantRace               category
ApplicantRa

### Check colums

In [38]:
df_selectdata_msa1.head()

,Year,RespondentID,AgencyCode,LoanType,PropertyType,LoanPurpose,Occupancy,LoanAmount,Preapproval,ActionType,...,Population,MinorityPopulationPercen,FFIECMedianFamily,TracttoMSA,NumberofOwnerOccupied,Numberof1_To,ApplicationDateIndicator,StateName,CountyName,CLASSCODEFIPS
0,2015,0000001281,3,1,1,3.0,1.0,222.0,3.0,3.0,...,4831,17.93,89600,117.52,1374,1797,0.0,AK,Matanuska-Susitna Borough,H1
1,2015,0000001281,3,1,1,3.0,2.0,188.0,3.0,3.0,...,5364,16.63,89600,111.23,1593,1754,0.0,AK,Matanuska-Susitna Borough,H1
2,2015,0000001281,3,1,1,3.0,2.0,216.0,3.0,1.0,...,1988,52.31,89600,58.72,149,327,0.0,AK,Anchorage Municipality,H6
3,2015,0000001281,3,1,1,3.0,1.0,219.0,3.0,1.0,...,3458,17.35,89600,112.09,998,1268,0.0,AK,Matanuska-Susitna Borough,H1
4,2015,0000001281,3,1,1,3.0,2.0,188.0,3.0,1.0,...,5364,16.63,89600,111.23,1593,1754,0.0,AK,Matanuska-Susitna Borough,H1


In [39]:
df_alldata=removeDatatForNullColumns(df_selectdata_msa1)
#df_readdata.info()

no item to drop


In [40]:
print(len(df_readdata))
print(len(df_selectdata_msa1))
print(len(df_alldata))

31630026
31630026
31630026


### Pickle and save the single file for further processing 

In [41]:
#df_selecteddata.info()
picklefilename='df_alldata_hmda_cenus.sa'
# get the pickle file name to re confirm the previous files are not overriden.
# Note: Run this file after confimring succesful completion of all above steps
print(picklefilename)

In [43]:
# create pickle file for further use 
pickle.dump(df_alldata,open(picklefilename,'wb'), protocol=4)

In [44]:
# check pickle file
upickle_df = pickle.load(open(picklefilename,"rb"))

In [45]:
upickle_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 48 columns):
Year                        int64
RespondentID                object
AgencyCode                  int64
LoanType                    int64
PropertyType                category
LoanPurpose                 category
Occupancy                   category
LoanAmount                  float64
Preapproval                 float64
ActionType                  float64
MSA                         float64
StateCode                   float64
CountyCode                  float64
CensusTractNumber           object
ApplicantEthnicity          category
CoApplicantEthnicity        category
ApplicantRace               category
ApplicantRace1              object
ApplicantRace2              object
ApplicantRace3              object
ApplicantRace4              object
CoApplicantRace             category
CoApplicantRace1            object
CoApplicantRace2            object
CoApplicantRace3            obj

In [46]:
df_newdata=upickle_df

In [47]:
df_newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 48 columns):
Year                        int64
RespondentID                object
AgencyCode                  int64
LoanType                    int64
PropertyType                category
LoanPurpose                 category
Occupancy                   category
LoanAmount                  float64
Preapproval                 float64
ActionType                  float64
MSA                         float64
StateCode                   float64
CountyCode                  float64
CensusTractNumber           object
ApplicantEthnicity          category
CoApplicantEthnicity        category
ApplicantRace               category
ApplicantRace1              object
ApplicantRace2              object
ApplicantRace3              object
ApplicantRace4              object
CoApplicantRace             category
CoApplicantRace1            object
CoApplicantRace2            object
CoApplicantRace3            obj

In [48]:
for col in df_newdata.columns:
    
    df=checkforNullCol(col,df_newdata)
    if (len(df)>0):
        print(col + ' Null len :'+str(len(df)) )
    #df1=checkforNanCol(col)
    #if (len(df1)>0):
        #print(col + ' Nan len :'+str(len(df1)) )
        

MSA Null len :3495091
CountyCode Null len :49639
CensusTractNumber Null len :44206
ApplicantRace1 Null len :10051708
ApplicantRace2 Null len :10111851
ApplicantRace3 Null len :10115618
ApplicantRace4 Null len :10115986
CoApplicantRace1 Null len :10094071
CoApplicantRace2 Null len :10115111
CoApplicantRace3 Null len :10116202
CoApplicantRace4 Null len :10116428
DenialReason1 Null len :8869448
DenialReason2 Null len :9863088
DenialReason3 Null len :10080546
RateSpread Null len :30317214
EditStatus Null len :10116644
SequenceNumber Null len :10116644
Population Null len :44208
MinorityPopulationPercen Null len :44228
FFIECMedianFamily Null len :44208
TracttoMSA Null len :44208
NumberofOwnerOccupied Null len :44228
Numberof1_To Null len :44228
ApplicationDateIndicator Null len :10116644
CountyName Null len :49682
CLASSCODEFIPS Null len :49682


In [49]:
df_newdata.groupby('ActionType').size() 

ActionType
1.0    21287511
2.0    1125929 
3.0    6450624 
6.0    2762256 
7.0    1952    
8.0    1754    
dtype: int64

In [50]:
print(len(df_newdata))
print(len(df_alldata))

31630026
31630026


In [3]:
### rough

In [7]:
df_readdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31630026 entries, 0 to 49681
Data columns (total 48 columns):
Year                        int64
RespondentID                object
AgencyCode                  int64
LoanType                    int64
PropertyType                category
LoanPurpose                 category
Occupancy                   category
LoanAmount                  float64
Preapproval                 float64
ActionType                  float64
MSA                         float64
StateCode                   float64
CountyCode                  float64
CensusTractNumber           object
ApplicantEthnicity          category
CoApplicantEthnicity        category
ApplicantRace               category
ApplicantRace1              object
ApplicantRace2              object
ApplicantRace3              object
ApplicantRace4              object
CoApplicantRace             category
CoApplicantRace1            object
CoApplicantRace2            object
CoApplicantRace3            obj

In [19]:
t=df_readdata[(df_readdata.ApplicantRace1.notna()) & (df_readdata.ApplicantRace1!='') & (df_readdata.ApplicantRace1!=' ') & (df_readdata.ApplicantRace!=df_readdata.ApplicantRace1)]


In [27]:
#t.loc[:,['ApplicantRace1','ApplicantRace']]
#t.ApplicantRace1.unique()
t.groupby(['ApplicantRace1','ApplicantRace']).count()

Year  RespondentID  AgencyCode  LoanType  \
ApplicantRace1 ApplicantRace                                                
1.0            1.0           NaN      NaN           NaN         NaN         
               2.0            70.0     70.0          70.0        70.0       
               3.0            299.0    299.0         299.0       299.0      
               4.0            22.0     22.0          22.0        22.0       
               5.0            1995.0   1995.0        1995.0      1995.0     
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
2.0            1.0            1575.0   1575.0        1575.0      1575.0     
               2.0           NaN      NaN           NaN         NaN         
               3.0            159.0    159.0         159.0       159.0      
               4.0            203.0    203.0         203.0       203.0      
               5.0            1488.0   1488.0        1488.0      1488.0     
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
3.0            1.0            4527.0   4527.0        4527.0      4527.0     
               2.0            1625.0   1625.0        1625.0      1625.0     
               3.0           NaN      NaN           NaN         NaN         
               4.0            67.0     67.0          67.0        67.0       
               5.0            1026.0   1026.0        1026.0      1026.0     
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
4.0            1.0            405.0    405.0         405.0       405.0      
               2.0            3231.0   3231.0        3231.0      3231.0     
               3.0            916.0    916.0         916.0       916.0      
               4.0           NaN      NaN           NaN         NaN         
               5.0            677.0    677.0         677.0       677.0      
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
5.0            1.0            18668.0  18668.0       18668.0     18668.0    
               2.0            12505.0  12505.0       12505.0     12505.0    
...                               ...      ...           ...         ...    
1              6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
2              1.0            2359.0   2359.0        2359.0      2359.0     
               2.0           NaN      NaN           NaN         NaN         
               3.0            262.0    262.0         262.0       262.0      
               4.0            484.0    484.0         484.0       484.0      
               5.0            2863.0   2863.0        2863.0      2863.0     
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
3              1.0            7072.0   7072.0        7072.0      7072.0     
               2.0            2700.0   2700.0        2700.0      2700.0     
               3.0           NaN      NaN           NaN         NaN         
               4.0            148.0    148.0         148.0       148.0      
               5.0            2179.0   2179.0        2179.0      2179.0     
               6.0           NaN      NaN           NaN         NaN         
               7.0           NaN      NaN           NaN         NaN         
4              1.0            675.0    675.0         675.0       675.0      
               2.0            5781.0   5781.0        5781.0      5781.0     
               3.0            1490.0   1490.0        1490.0      1490.0     
               4.0           